# Step 1: Core Schemas v1 - Building One Table at a Time

## Why One File/Table at a Time?

In the SkinStack affiliate platform, each table represents a **critical business entity**:
- **Users** → Who can access the system
- **Influencers** → Who creates tracking links
- **Merchants** → Who pays commissions
- **Programs** → Commission rules
- **Products** → What's being sold
- **Tracking Links** → The core money-maker

Building one at a time ensures:
1. **No circular dependencies**
2. **Each table is tested before the next**
3. **Migrations are atomic and reversible**

## Table 1: USERS (Foundation)

### Why First?
Everything in SkinStack requires authentication. Without users, nothing else can exist.

### Business Purpose:
- Influencers need accounts to get paid
- Merchants need accounts to manage programs
- Admins need accounts to oversee platform

### Minimal Fields for MVP:
- `id` (UUID) - Unique identifier
- `email` - Login credential
- `password_hash` - Security
- `role` - Access control (influencer/merchant/admin)
- `created_at` - Audit trail

In [ ]:
# Migration 001_users.sql
sql_users = """
-- Migration 001: Users table (Foundation)
-- Purpose: Authentication and role-based access

CREATE TABLE users (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    email TEXT UNIQUE NOT NULL,
    password_hash TEXT NOT NULL,
    role TEXT CHECK(role IN ('influencer', 'merchant', 'admin')) NOT NULL,
    is_active BOOLEAN DEFAULT true,
    created_at TIMESTAMPTZ DEFAULT NOW(),
    updated_at TIMESTAMPTZ DEFAULT NOW()
);

-- Index for fast login lookups
CREATE INDEX idx_users_email ON users(email);
CREATE INDEX idx_users_role ON users(role) WHERE is_active = true;

-- Update trigger for updated_at
CREATE OR REPLACE FUNCTION update_updated_at()
RETURNS TRIGGER AS $$
BEGIN
    NEW.updated_at = NOW();
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER users_updated_at 
    BEFORE UPDATE ON users 
    FOR EACH ROW 
    EXECUTE FUNCTION update_updated_at();

-- Record migration
INSERT INTO schema_migrations (version, name) VALUES (1, 'users');
"""

print("✅ Users table SQL ready")
print(f"Lines: {len(sql_users.splitlines())}")

## Table 2: INFLUENCERS (Revenue Generators)

### Why Second?
Influencers are the primary revenue drivers. They need user accounts first (foreign key).

### Business Purpose:
- Store payout details (Stripe, PayPal)
- Track performance metrics
- Social media handles for verification

### Minimal Fields for MVP:
- Links to `users.id`
- Payment details
- Social proof (followers)

In [ ]:
# Migration 002_influencers.sql
sql_influencers = """
-- Migration 002: Influencers table
-- Purpose: Track influencer-specific data for payouts

CREATE TABLE influencers (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    user_id UUID UNIQUE NOT NULL REFERENCES users(id) ON DELETE CASCADE,
    display_name TEXT NOT NULL,
    instagram_handle TEXT,
    tiktok_handle TEXT,
    youtube_handle TEXT,
    
    -- Payout info
    stripe_account_id TEXT,
    paypal_email TEXT,
    preferred_payout_method TEXT CHECK(preferred_payout_method IN ('stripe', 'paypal', 'ach')),
    
    -- Metrics
    total_followers INTEGER DEFAULT 0,
    total_earned DECIMAL(10,2) DEFAULT 0,
    total_paid DECIMAL(10,2) DEFAULT 0,
    
    created_at TIMESTAMPTZ DEFAULT NOW(),
    updated_at TIMESTAMPTZ DEFAULT NOW()
);

-- Indexes for quick lookups
CREATE INDEX idx_influencers_user_id ON influencers(user_id);
CREATE INDEX idx_influencers_instagram ON influencers(instagram_handle) WHERE instagram_handle IS NOT NULL;

-- Trigger for updated_at
CREATE TRIGGER influencers_updated_at 
    BEFORE UPDATE ON influencers 
    FOR EACH ROW 
    EXECUTE FUNCTION update_updated_at();

-- Record migration
INSERT INTO schema_migrations (version, name) VALUES (2, 'influencers');
"""

print("✅ Influencers table SQL ready")
print(f"Lines: {len(sql_influencers.splitlines())}")

## Table 3: MERCHANTS (Money Source)

### Why Third?
Merchants pay the bills. They need user accounts and define commission programs.

### Business Purpose:
- Store business details
- Connect to Shopify/Refersion/Impact
- Track total spend

In [ ]:
# Migration 003_merchants.sql
sql_merchants = """
-- Migration 003: Merchants table
-- Purpose: Brands that offer affiliate programs

CREATE TABLE merchants (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    user_id UUID REFERENCES users(id) ON DELETE SET NULL,
    business_name TEXT NOT NULL,
    website_url TEXT NOT NULL,
    
    -- Integration credentials (encrypted in production)
    shopify_shop_domain TEXT,
    shopify_api_key TEXT,
    refersion_public_key TEXT,
    refersion_secret_key TEXT,
    
    -- Business metrics
    total_sales DECIMAL(12,2) DEFAULT 0,
    total_commissions_paid DECIMAL(10,2) DEFAULT 0,
    
    is_active BOOLEAN DEFAULT true,
    created_at TIMESTAMPTZ DEFAULT NOW(),
    updated_at TIMESTAMPTZ DEFAULT NOW()
);

-- Indexes
CREATE INDEX idx_merchants_active ON merchants(is_active);
CREATE INDEX idx_merchants_shopify ON merchants(shopify_shop_domain) WHERE shopify_shop_domain IS NOT NULL;

-- Trigger
CREATE TRIGGER merchants_updated_at 
    BEFORE UPDATE ON merchants 
    FOR EACH ROW 
    EXECUTE FUNCTION update_updated_at();

-- Record migration
INSERT INTO schema_migrations (version, name) VALUES (3, 'merchants');
"""

print("✅ Merchants table SQL ready")

## Table 4: PROGRAMS (Commission Rules)

### Why Fourth?
Programs define HOW MUCH influencers earn. Need merchants first.

### Business Purpose:
- Set commission rates (15-30% typical for skincare)
- Define cookie windows (7-30 days)
- Control which networks (Shopify, Impact, etc.)

In [ ]:
# Migration 004_programs.sql
sql_programs = """
-- Migration 004: Programs table
-- Purpose: Define commission structures and rules

CREATE TABLE programs (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    merchant_id UUID NOT NULL REFERENCES merchants(id) ON DELETE CASCADE,
    name TEXT NOT NULL,
    network TEXT NOT NULL CHECK(network IN ('shopify_refersion', 'impact', 'amazon', 'levanta')),
    
    -- Commission structure
    commission_type TEXT CHECK(commission_type IN ('percentage', 'fixed')) DEFAULT 'percentage',
    commission_rate DECIMAL(5,2) CHECK(commission_rate > 0 AND commission_rate <= 100),
    fixed_amount DECIMAL(10,2),
    
    -- Attribution settings
    cookie_window_days INTEGER DEFAULT 30,
    
    -- Platform fee (SkinStack takes 20%)
    platform_fee_rate DECIMAL(3,2) DEFAULT 0.20,
    
    is_active BOOLEAN DEFAULT true,
    created_at TIMESTAMPTZ DEFAULT NOW(),
    updated_at TIMESTAMPTZ DEFAULT NOW(),
    
    CONSTRAINT valid_commission CHECK (
        (commission_type = 'percentage' AND commission_rate IS NOT NULL) OR
        (commission_type = 'fixed' AND fixed_amount IS NOT NULL)
    )
);

-- Indexes
CREATE INDEX idx_programs_merchant ON programs(merchant_id);
CREATE INDEX idx_programs_network ON programs(network);
CREATE INDEX idx_programs_active ON programs(is_active);

-- Trigger
CREATE TRIGGER programs_updated_at 
    BEFORE UPDATE ON programs 
    FOR EACH ROW 
    EXECUTE FUNCTION update_updated_at();

INSERT INTO schema_migrations (version, name) VALUES (4, 'programs');
"""

print("✅ Programs table SQL ready")

## Table 5: PRODUCTS (What's Being Sold)

### Why Fifth?
Products belong to merchants and programs. Links will point to products.

### Business Purpose:
- Track skincare products (serums, cleansers, etc.)
- Store images for link previews
- Calculate commissions per product

In [ ]:
# Migration 005_products.sql
sql_products = """
-- Migration 005: Products table
-- Purpose: Skincare products available for promotion

CREATE TABLE products (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    merchant_id UUID NOT NULL REFERENCES merchants(id) ON DELETE CASCADE,
    program_id UUID REFERENCES programs(id) ON DELETE SET NULL,
    
    -- Product details
    external_id TEXT NOT NULL, -- Shopify/Amazon product ID
    name TEXT NOT NULL,
    description TEXT,
    price DECIMAL(10,2) NOT NULL,
    image_url TEXT,
    product_url TEXT NOT NULL,
    
    -- Categories for skincare
    category TEXT CHECK(category IN (
        'cleanser', 'serum', 'moisturizer', 'sunscreen', 
        'mask', 'toner', 'eye_cream', 'treatment', 'set'
    )),
    
    -- Inventory
    in_stock BOOLEAN DEFAULT true,
    
    created_at TIMESTAMPTZ DEFAULT NOW(),
    updated_at TIMESTAMPTZ DEFAULT NOW(),
    
    UNIQUE(merchant_id, external_id)
);

-- Indexes
CREATE INDEX idx_products_merchant ON products(merchant_id);
CREATE INDEX idx_products_program ON products(program_id);
CREATE INDEX idx_products_external ON products(external_id);
CREATE INDEX idx_products_category ON products(category);

-- Trigger
CREATE TRIGGER products_updated_at 
    BEFORE UPDATE ON products 
    FOR EACH ROW 
    EXECUTE FUNCTION update_updated_at();

INSERT INTO schema_migrations (version, name) VALUES (5, 'products');
"""

print("✅ Products table SQL ready")

## Table 6: TRACKING_LINKS (The Money Maker! 💰)

### Why Last?
Links need ALL previous tables:
- Influencer (who's promoting)
- Program (commission rules)
- Product (what's being sold)

### Business Purpose:
- Generate unique short URLs (skin.st/AbC123)
- Track clicks → conversions → commissions
- The CORE of the affiliate business

In [ ]:
# Migration 006_tracking_links.sql
sql_tracking_links = """
-- Migration 006: Tracking Links table
-- Purpose: The CORE table - tracks affiliate links

CREATE TABLE tracking_links (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    
    -- The unique short slug (e.g., 'abc123' for skin.st/abc123)
    slug TEXT UNIQUE NOT NULL,
    
    -- Relationships
    influencer_id UUID NOT NULL REFERENCES influencers(id) ON DELETE CASCADE,
    program_id UUID NOT NULL REFERENCES programs(id) ON DELETE CASCADE,
    product_id UUID REFERENCES products(id) ON DELETE SET NULL,
    
    -- Where it goes
    destination_url TEXT NOT NULL,
    
    -- Campaign tracking
    campaign_code TEXT,
    utm_source TEXT DEFAULT 'skinstack',
    utm_medium TEXT DEFAULT 'affiliate',
    utm_campaign TEXT,
    
    -- Performance metrics (denormalized for speed)
    total_clicks INTEGER DEFAULT 0,
    total_conversions INTEGER DEFAULT 0,
    total_revenue DECIMAL(10,2) DEFAULT 0,
    total_commission DECIMAL(10,2) DEFAULT 0,
    
    -- Status
    is_active BOOLEAN DEFAULT true,
    expires_at TIMESTAMPTZ,
    
    created_at TIMESTAMPTZ DEFAULT NOW(),
    updated_at TIMESTAMPTZ DEFAULT NOW()
);

-- CRITICAL: Index on slug for fast redirects (must be < 20ms)
CREATE UNIQUE INDEX idx_tracking_links_slug ON tracking_links(slug);

-- Other indexes
CREATE INDEX idx_tracking_links_influencer ON tracking_links(influencer_id);
CREATE INDEX idx_tracking_links_program ON tracking_links(program_id);
CREATE INDEX idx_tracking_links_active ON tracking_links(is_active) WHERE is_active = true;
CREATE INDEX idx_tracking_links_campaign ON tracking_links(campaign_code) WHERE campaign_code IS NOT NULL;

-- Function to generate unique slug
CREATE OR REPLACE FUNCTION generate_unique_slug()
RETURNS TEXT AS $$
DECLARE
    new_slug TEXT;
    done BOOLEAN DEFAULT false;
BEGIN
    WHILE NOT done LOOP
        -- Generate 6-character slug
        new_slug := substr(md5(random()::text), 1, 6);
        
        -- Check if exists
        IF NOT EXISTS (SELECT 1 FROM tracking_links WHERE slug = new_slug) THEN
            done := true;
        END IF;
    END LOOP;
    
    RETURN new_slug;
END;
$$ LANGUAGE plpgsql;

-- Trigger
CREATE TRIGGER tracking_links_updated_at 
    BEFORE UPDATE ON tracking_links 
    FOR EACH ROW 
    EXECUTE FUNCTION update_updated_at();

INSERT INTO schema_migrations (version, name) VALUES (6, 'tracking_links');
"""

print("✅ Tracking Links table SQL ready")
print("\n🎯 This is the MONEY TABLE!")

## Execution Plan - One Migration at a Time

### Why This Order Matters:

1. **Users** → Foundation (no dependencies)
2. **Influencers** → Needs users (1 dependency)
3. **Merchants** → Needs users (1 dependency)
4. **Programs** → Needs merchants (1 dependency)
5. **Products** → Needs merchants & programs (2 dependencies)
6. **Tracking Links** → Needs everything (4 dependencies)

Each migration:
- Can be rolled back independently
- Is tested before the next
- Adds business value incrementally

In [ ]:
# Create the combined migration file
combined_sql = f"""
-- SkinStack Core Schema v1
-- Execute one migration at a time!

{sql_users}

-- After users, test login works, then continue...

{sql_influencers}

-- After influencers, test profile creation, then continue...

{sql_merchants}

-- After merchants, test business onboarding, then continue...

{sql_programs}

-- After programs, test commission setup, then continue...

{sql_products}

-- After products, test product import, then continue...

{sql_tracking_links}

-- DONE! The money-making machine is ready!
"""

# Save to file
with open('sql/migrations/001_core.sql', 'w') as f:
    f.write(combined_sql)

print("✅ Migration file created: sql/migrations/001_core.sql")
print(f"Total lines: {len(combined_sql.splitlines())}")
print("\n🚀 Ready to build SkinStack one table at a time!")